In [1]:
import yfinance as yf

In [2]:
each_symbol = input()
df = yf.Ticker(f'{each_symbol}.NS').history("1y")

 SBIN


In [4]:
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-12-04 00:00:00+05:30,588.000000,596.000000,584.549988,594.700012,26077882,0.0,0.0
2023-12-05 00:00:00+05:30,595.049988,611.299988,595.049988,608.250000,36430152,0.0,0.0
2023-12-06 00:00:00+05:30,614.000000,614.000000,604.549988,608.450012,21439069,0.0,0.0
2023-12-07 00:00:00+05:30,610.000000,612.549988,605.000000,611.700012,10725706,0.0,0.0
2023-12-08 00:00:00+05:30,610.000000,616.799988,605.349976,614.150024,17722136,0.0,0.0


In [5]:
import pandas_ta as ta
def append_sma(df,**kwargs):
  """
  Calculates and appends the Simple Moving Average (SMA) to a DataFrame.

  Args:
      df: The DataFrame where the SMA will be calculated.
      column: Name of the column to use for calculating the SMA (default: "Close").
      length: Number of periods for the SMA calculation (default: 20).
      offset: Number of periods to shift the SMA forward (positive) or backward (negative) (default: 0).
      suffix: Suffix to add to the new SMA column name (default: "_SMA").
      inplace: Whether to modify the original DataFrame or return a new one (default: False).

  Returns:
      A DataFrame with the SMA appended as a new column.

  Raises:
      ValueError: If an invalid offset is provided.
  """

  column = kwargs.get("column", "close")
  length = kwargs.get("length", 50)
  offset = kwargs.get("offset", 0)

  if offset < 0:
      raise ValueError("Offset for SMA cannot be negative.")

  df.ta.sma(close=column, length=length, offset=offset,append=True)

  return df

def evaluate_sma(df, resultlist, **kwargs):
  """
  Evaluates the conditions and appends the DataFrame's symbol_id to a list of DataFrames if they meet the conditions.

  Args:
      df: The DataFrame to be evaluated.
      resultlist: A list of DataFrames where the symbol_id will be appended if the conditions are met.
      kwargs: A dictionary containing the following keys:
          - direction: "increasing" or "decreasing"
          - condition: "minimum", "maximum", or "exact"
          - bars: Number of bars to consider for the condition
          - additional_kwargs: Any additional keyword arguments passed to the function

  Returns:
      A list of DataFrames with the symbol_id appended if the conditions were met.
  """
  # Extract the conditions from kwargs
  conditions = {
      "direction": kwargs.get("direction"),
      "condition": kwargs.get("condition"),
      "bars": kwargs.get("bars"),
  }

# Check if the expected SMA column already exists
  sma_name = kwargs.get("name", "SMA") + "_" + str(kwargs.get("length"))
  if sma_name not in df.columns:
      raise ValueError(f"SMA column '{sma_name}' not found in the DataFrame.")

  # Use the existing SMA column for evaluation
  df['SMA'] = df[sma_name]

  # Check the direction
  if conditions['direction'] == "increasing":
      trend = df['SMA'] > df['SMA'].shift(1)
  else:
      trend = df['SMA'] < df['SMA'].shift(1)


  if conditions['condition'] == "minimum":
      minimum = df['SMA'].rolling(window=conditions['bars']).min()
      condition = df['SMA'] == minimum
  elif conditions['condition'] == "maximum":
      maximum = df['SMA'].rolling(window=conditions['bars']).max()
      condition = df['SMA'] == maximum
  else:
      # Check if SMA is increasing for exactly 5 bars
      condition = trend.iloc[-int(conditions['bars']):] # type: ignore

  # Check if the conditions are met
  if condition.iloc[-1]:
      print("TRUE")
      resultlist.add(df["symbol_id"][0])
  return resultlist


def append_macd(df, column="Close", fast_length=12, slow_length=26, signal_length=9, offset=0, suffix="_MACD", inplace=True):

    """
    Calculates and appends the Moving Average Convergence Divergence (MACD) to a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame where the MACD will be calculated.
        column (str): Name of the column to use for calculating the MACD (default: "Close").
        fast_length (int): Number of periods for the fast EMA (default: 12).
        slow_length (int): Number of periods for the slow EMA (default: 26).
        signal_length (int): Number of periods for the MACD signal line (default: 9).
        offset (int): Number of periods to shift the MACD forward (positive) or backward (negative) (default: 0).
        suffix (str): Suffix to add to the new MACD column names (default: "_MACD").
        inplace (bool): Whether to modify the original DataFrame or return a new one (default: True).

    Returns:
        pandas.DataFrame: The DataFrame with the MACD appended as new columns (in-place modification).
    """

    if not inplace:
        raise ValueError("The `append_macd` function only supports in-place modification (inplace=True).")

    # Calculate the MACD components and the MACD signal line
    macd = df.ta.macd(close=column, fast_length=fast_length, slow_length=slow_length, signal_length=signal_length, offset=offset,append=True)
    


In [10]:
indicators = {
            # "name":"SMA",
            # "column": "close",
            # "length": 44,
            # "offset": 0,
            # "direction":"increasing",
            # "condition":"minimum",
            # "bars":3
        }


In [11]:
append_macd(df,**indicators)

In [23]:
result = set()
ss = evaluate_sma(df,result,**indicators)

In [12]:
df.tail(50)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,,,,,,,,
2023-09-26 00:00:00+05:30,594.000000,595.000000,591.250000,594.400024,8327427,0.0,0.0,4.833690,1.053295,3.780395
2023-09-27 00:00:00+05:30,593.450012,594.299988,588.200012,589.750000,9865347,0.0,0.0,4.266248,0.388682,3.877566
2023-09-28 00:00:00+05:30,591.250000,595.200012,588.849976,589.950012,21960816,0.0,0.0,3.789007,-0.070847,3.859854
2023-09-29 00:00:00+05:30,592.849976,600.000000,591.299988,598.549988,12189735,0.0,0.0,4.057959,0.158484,3.899475
2023-10-03 00:00:00+05:30,596.599976,604.900024,589.599976,602.950012,15322196,0.0,0.0,4.573431,0.539165,4.034266
2023-10-04 00:00:00+05:30,600.000000,600.450012,584.450012,586.250000,24914612,0.0,0.0,3.592977,-0.353031,3.946008
2023-10-05 00:00:00+05:30,590.000000,594.349976,587.099976,592.150024,13248028,0.0,0.0,3.254527,-0.553185,3.807712
2023-10-06 00:00:00+05:30,593.400024,598.950012,592.200012,594.250000,8216780,0.0,0.0,3.119790,-0.550337,3.670128
2023-10-09 00:00:00+05:30,588.000000,589.000000,581.549988,585.099976,9189597,0.0,0.0,2.248758,-1.137096,3.385854


In [25]:
print(ss)

set()
